In [1]:
%load_ext autoreload
%autoreload 2
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import logging
logging.basicConfig(level=logging.INFO)

import pyrosetta
import pyrosetta.distributed.io as io
import pyrosetta.distributed.packed_pose as packed_pose
import pyrosetta.distributed.tasks.rosetta_scripts as rosetta_scripts
import pyrosetta.distributed.tasks.score as score

import pandas as pd
import pandas

import numpy as np
from glob import glob
import os

In [2]:
import truncator

In [3]:
print(pyrosetta.version())

PyRosetta-4 2019 [Rosetta devel 2019.01+HEAD.dbc838b6ae620b1293476b1bd4366ffc2facc5b5 2019-01-03T10:31:13] retrieved from: ssh://git@github.com/RosettaCommons/main
(C) Copyright Rosetta Commons Member Institutions.
Created in JHU by Sergey Lyskov and PyRosetta Team.



In [4]:
import sys; sys.path.insert(0, "/home/ajasja/projects/Stable_Heterodimers")
import utils as u

In [5]:
ON_CLUSTER=False

In [6]:
flags = """
-beta \
-renumber_pdb 0 \
-overwrite 1 \
-out:pdb \
-holes:dalphaball truncator/scripts/DAlphaBall.gcc 
"""

In [7]:
if not ON_CLUSTER:
    pyrosetta.init(flags)
else:
    cluster = u.init_env(flags, 10)

INFO:pyrosetta.rosetta:Found rosetta database at: /software/conda/envs/pyrosetta/lib/python3.7/site-packages/pyrosetta/database; using it....
INFO:pyrosetta.rosetta:PyRosetta-4 2019 [Rosetta devel 2019.01+HEAD.dbc838b6ae620b1293476b1bd4366ffc2facc5b5 2019-01-03T10:31:13] retrieved from: ssh://git@github.com/RosettaCommons/main
(C) Copyright Rosetta Commons Member Institutions.
Created in JHU by Sergey Lyskov and PyRosetta Team.

INFO:rosetta:core.init: Checking for fconfig files in pwd and ./rosetta/flags
INFO:rosetta:core.init.score_function_corrections: [ WARNING ] Flag -beta_nov16 is set but -weights are also specified.  Not changing input weights file!
INFO:rosetta:core.init: Rosetta version: 2019.01+HEAD.dbc838b6ae6 dbc838b6ae620b1293476b1bd4366ffc2facc5b5 ssh://git@github.com/RosettaCommons/main 2019-01-03T10:31:13
INFO:rosetta:core.init: command: PyRosetta -beta -renumber_pdb 0 -overwrite 1 -out:pdb -holes:dalphaball truncator/scripts/DAlphaBall.gcc -database /software/conda/env

In [8]:
def lines_grep(pattern, lines):
    import re
    reg = re.compile(pattern)
    return [l for l in lines if reg.match(l)]

def print_sele(scores, name, color=None, remove_chains=False):
    sele = scores[name+"_pymol_selection"]
    sele=sele.replace("rosetta_sele", name)
    if remove_chains:
        sele=sele.replace("chain A and", "").replace("chain B and", "")
    print(sele)
    if not color is None:
        print(f"color {color}, {name}")
    

In [16]:
%%time
#Pyrosetta unsupported BuriedUnsatHbonds2 InterfaceHoles CavityVolume
pack_protocol = truncator.read_file("truncator/xml/05_specific_interface_markers.xml")
#print(lines_grep(".*Neighborhood.*", pack_protocol.split("\n")))
task_design = rosetta_scripts.SingleoutputRosettaScriptsTask(pack_protocol)
# Syntax check via setup
logging.basicConfig(level=logging.WARN)
task_design.setup()
raw_input_pose = io.pose_from_file(filename=r"data\ZCON_37_N3_C1_2.pdb".replace("\\","/"))
output_pose = task_design(raw_input_pose)

print("color deeppurple, all")#rint_sele(output_pose.scores, "core_bndr_Hbonds", "yellow")
output_pose.scores

INFO:rosetta:protocols.rosetta_scripts.RosettaScriptsParser: Validating input script...
INFO:rosetta:protocols.rosetta_scripts.RosettaScriptsParser: ...done
INFO:rosetta:protocols.rosetta_scripts.RosettaScriptsParser: Parsed script:
<ROSETTASCRIPTS>
	<SCOREFXNS>
		<ScoreFunction name="NOV15" weights="beta">
			<Reweight scoretype="coordinate_constraint" weight="1"/>
		</ScoreFunction>
		<ScoreFunction name="VDW" weights="empty">
			<Reweight scoretype="fa_atr" weight="1.0"/>
		</ScoreFunction>
		<ScoreFunction name="hpatch" weights="empty">
			<Reweight scoretype="hpatch" weight="1.0"/>
		</ScoreFunction>
	</SCOREFXNS>
	<RESIDUE_SELECTORS>
		<Layer core_cutoff="3.6" name="core" select_core="true" surface_cutoff="2.0" use_sidechain_neighbors="true"/>
		<Layer core_cutoff="3.6" name="bndr" select_boundary="true" surface_cutoff="2.0" use_sidechain_neighbors="true"/>
		<Layer core_cutoff="3.6" name="surf" select_surface="true" surface_cutoff="2.0" use_sidechain_neighbors="true"/>
		<Layer 

color deeppurple, all
CPU times: user 17.3 s, sys: 424 ms, total: 17.7 s
Wall time: 18.4 s


In [17]:
s =output_pose.scores; s

{'all_buns_bb': 1.0,
 'all_buns_sc': 2.0,
 'chA_SASA': 2896.662841796875,
 'chA_exposed_hydrophobics': 647.7210083007812,
 'chA_hpatch_score': 100.0,
 'chA_hydrophobic_SASA': 1989.284423828125,
 'chA_num_core_SASA_polars': 3.0,
 'chA_polar_SASA': 907.3784790039062,
 'chB_SASA': 6698.357421875,
 'chB_exposed_hydrophobics': 821.889892578125,
 'chB_hpatch_score': 48.959999084472656,
 'chB_hydrophobic_SASA': 3944.41162109375,
 'chB_num_core_SASA_polars': 3.0,
 'chB_polar_SASA': 2753.94580078125,
 'complex_normalized': -3.036371946334839,
 'core_buns_bb': 1.0,
 'core_buns_sc': 2.0,
 'dG_cross': -102.26443481445312,
 'dG_cross/dSASAx100': -3.979475259780884,
 'dG_separated': -94.46307373046875,
 'dG_separated/dSASAx100': -3.675896406173706,
 'dSASA_hphobic': 1993.1282958984375,
 'dSASA_int': 2569.796875,
 'dSASA_polar': 576.668701171875,
 'ddg': -95.59739685058594,
 'delta_unsatHbonds': 5.0,
 'hbond_E_fraction': 0.08154125511646271,
 'hbonds_int': 7.0,
 'interface_hbonds': 7.0,
 'interface_s

In [19]:
truncator.write_file("test_1.pdb", io.to_pdbstring(output_pose))

In [20]:
print("color deeppurple, all")
#print_sele(output_pose.scores, "hbnet_residues", "yellow")
print_sele(output_pose.scores, "core_bndr_polars", "orange")
print_sele(output_pose.scores, "core_SASA_polars", "red")


color deeppurple, all
select core_bndr_polars, (chain A and resid 9,16,17,22) or (chain B and resid 34,38,41,45,58,68,71,81,117,118)
color orange, core_bndr_polars
select core_SASA_polars, (chain A and resid 9,16,17) or (chain B and resid 38,45,68,71)
color red, core_SASA_polars


In [78]:
print("color deeppurple, all")
print_sele(output_pose.scores, "hbnet_residues", "blue")
print_sele(output_pose.scores, "core_hbnet_residues_chA", "yellow")
print_sele(output_pose.scores, "core_hbnet_residues_chB", "orange")
#print_sele(output_pose.scores, "core_SASA", "red")

color deeppurple, all
select hbnet_residues, (chain A and resid 19,26,29,50,53) or (chain B and resid 99,106,123,130)
color blue, hbnet_residues
select core_hbnet_residues_chA, (chain A and resid 19,26,50,53)
color yellow, core_hbnet_residues_chA
select core_hbnet_residues_chB, (chain B and resid 123,130)
color orange, core_hbnet_residues_chB


In [31]:

print("color deeppurple, all")
print_sele(output_pose.scores, "interface_by_vec", "orange")
print_sele(output_pose.scores, "core_bndr_and_interface_by_vec", "red")

color deeppurple, all
select interface_by_vec, (chain A and resid 8,9,11,12,15,16,19,22,23,26,29,30,32,33,42,43,45,46,49,50,52,53,56,57,59,60,63,64,67) or (chain B and resid 88,89,91,92,95,96,98,99,102,103,105,106,109,112,113,120,123,124,127,130,131,134,135,137,138,141,142,144)
color orange, interface_by_vec
select core_bndr_and_interface_by_vec, (chain A and resid 12,15,16,19,22,23,26,29,30,33,42,43,46,49,50,53,56,57,60,63,64,67) or (chain B and resid 88,91,95,98,102,105,106,109,112,113,120,123,127,130,131,134,137,138,141,144)
color red, core_bndr_and_interface_by_vec


In [20]:
print("color deeppurple, all")
print_sele(output_pose.scores, "interface_by_vec", "orange")
print_sele(output_pose.scores, "core_bndry_and_interface_by_vec", "red")

color deeppurple, all
select interface_by_vec, (chain A and resid 8,9,11,12,15,16,19,22,23,26,29,30,32,33,42,43,45,46,49,50,52,53,56,57,59,60,63,64,67) or (chain B and resid 88,89,91,92,95,96,98,99,102,103,105,106,109,112,113,120,123,124,127,130,131,134,135,137,138,141,142,144)
color orange, interface_by_vec


KeyError: 'core_bndry_and_interface_by_vec_pymol_selection'

NameError: name 'truncator' is not defined

In [1]:
output_pose.scores

NameError: name 'output_pose' is not defined

In [15]:
score_file = "test.sc" 
struct_name = "data/ZCON_37_N3_C1_2.pdb"
!/software/rosetta/latest/bin/rosetta_scripts -parser:protocol truncator/xml/80_test_selectors.xml -s {struct_name} \
    -out:file:scorefile  {score_file} \
    -print_pymol_selection 0 \
    -beta \
    -in:file:fullatom \
    -renumber_pdb 1 \
    -overwrite \
    -out:no_nstruct_label \
    -out:file:scorefile_format json > /dev/null
      

In [37]:
                          
scores = pd.read_json(score_file, lines=True).iloc[-1].to_dict()

In [47]:
print("color deeppurple, all")
print_sele(scores, "chainA", "orange")
print_sele(scores, "chainB", "red")

color deeppurple, all
select chainA, ( resid 92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119)
color orange, chainA
select chainB, ( resid 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91)
color red, chainB


In [19]:
print("color deeppurple, all")
print_sele(scores, "interface_by_vec", "orange")
print_sele(scores, "core_and_interface_by_vec", "red")

color deeppurple, all


NameError: name 'scores' is not defined